# Jintao Lai (jl3837), Jessie Han, Jiaxin Guo

# Preparation 

In [2]:
import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from scipy import sparse
from heapq import *

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


stemmer = PorterStemmer()
lemm = WordNetLemmatizer()

training_set = 10000
test_set = 2000

[nltk_data] Downloading package stopwords to /Users/Terry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Terry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Terry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /Users/Terry/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Terry/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def get_time_stamp():
    now = int(time.time()) 
    timeArray = time.localtime()
    otherStyleTime = time.strftime("%Y-%m-%d_%H-%M-%S", timeArray)
    print otherStyleTime
    return otherStyleTime

from sklearn.preprocessing import normalize, FunctionTransformer
transformer = FunctionTransformer(np.log1p, validate=True)
def postprocessing(data):
#     normalize(data, norm='max', copy=False)
    data = transformer.transform(data)
    return data

# Load Tags

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import preprocessing



def get_tags(train_or_test):
    size = None
    path = None
    
    if train_or_test == "train":
        size = training_set
        path = 'tags_train/'
    elif  train_or_test == "test":
        size = test_set
        path = 'tags_test/' 
        

    tags = []
    for i in range(size):
        data = open(path + str(i) + '.txt')
        lines = data.readlines()
        temp = ""
        for line in lines:
            word = line.split(':')[1].replace(' ', '').split('\n')[0]
            temp+= (word+" ")

        tags.append(temp[0:-1])
    
    return tags



In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import preprocessing



def get_tags(train_or_test):
    size = None
    path = None
    
    if train_or_test == "train":
        size = training_set
        path = 'tags_train/'
    elif  train_or_test == "test":
        size = test_set
        path = 'tags_test/' 
        

    tags = []
    for i in range(size):
        data = open(path + str(i) + '.txt')
        lines = data.readlines()
        temp = ""
        for line in lines:
            word = line.split(':')[1].replace(' ', '').split('\n')[0]
            temp+= (word+" ")

        tags.append(temp[0:-1])
    
    return tags



In [6]:

train_tag = get_tags("train")
test_tag =  get_tags("test")

#print train_tag
# 空格
tag_vectorizer = CountVectorizer(stop_words='english',analyzer='word', ngram_range=(1, 1))
tag_vectorizer.fit(train_tag)

train_tag_vectors = tag_vectorizer.transform(train_tag)
print len(tag_vectorizer.vocabulary_)

test_tag_vectors = tag_vectorizer.transform(test_tag)

training_lables = train_tag_vectors.toarray().tolist()
test_lables = test_tag_vectors.toarray().tolist()


80


# Load Description

In [7]:

from sklearn.metrics import classification_report
from sklearn import svm 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

dictionary_description = {}

stop_words = set(stopwords.words('english'))

stemmer = SnowballStemmer("english")

def get_all_lines(train_or_test, word_tag):
    
    size = None
    path = None
    
    if train_or_test == "train":
        size = training_set
        path = 'descriptions_train/'
    elif  train_or_test == "test":
        size = test_set
        path = 'descriptions_test/' 
        
    i = 0
    all_lines =[]
    for i in range(size):
        data = open(path + str(i) + '.txt')
        lines = data.readlines()
        words = []
        for line in lines:
            text_list = nltk.word_tokenize(line)
            english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
            text_list = [word for word in text_list if word not in english_punctuations]
            stops = set(stopwords.words("english"))
            text_list = [word for word in text_list if word not in stops]
            text_list = nltk.pos_tag(text_list)
            
            word_set = None
            NN = set(["NN","NNS","NNPS","NNP"])
            JJ = set(["JJ","JJR","JJS"])
            VV = set(["VB","VBD","VBG","VBN","VBP","VBZ"])
            
            if word_tag == 1:
                word_set =  NN | JJ
            elif word_tag == 2:
                word_set = NN | JJ | VV
                #word_set =  JJ | VV
            else:
                word_set = NN
                
            for word,tag in text_list:
                if tag in word_set:
                    words.append(stemmer.stem(word))

        if i == 0:
            print words

        all_lines.append(" ".join(words))
    return all_lines

In [8]:
all_lines = get_all_lines("train",1)
print(all_lines[0])

all_lines_test = get_all_lines("test",1)
print all_lines_test[0]

[u'skateboard', u'show', u'picnic', u'tabl', u'stage', u'skateboard', u'trick', u'top', u'picnic', u'tabl', u'man', u'skateboard', u'top', u'tabl', u'skate', u'boarder', u'trick', u'picnic', u'tabl', u'person', u'skateboard', u'picnic', u'tabl', u'crowd', u'watch']
skateboard show picnic tabl stage skateboard trick top picnic tabl man skateboard top tabl skate boarder trick picnic tabl person skateboard picnic tabl crowd watch
[u'woman', u'street', u'doorway', u'woman', u'past', u'doorway', u'sidewalk', u'woman', u'cell', u'phone', u'check', u'woman', u'talk', u'phone', u'check', u'sidewalk', u'woman', u'blue', u'phone', u'sidewalk', u'front', u'build', u'black', u'planter', u'side', u'entranc']
woman street doorway woman past doorway sidewalk woman cell phone check woman talk phone check sidewalk woman blue phone sidewalk front build black planter side entranc


In [9]:
all_lines_p = get_all_lines("train",2)
print(all_lines_p[0])

all_lines_test_p = get_all_lines("test",2)
print all_lines_test_p[0]

[u'skateboard', u'put', u'show', u'use', u'picnic', u'tabl', u'stage', u'skateboard', u'pull', u'trick', u'top', u'picnic', u'tabl', u'man', u'ride', u'skateboard', u'top', u'tabl', u'skate', u'boarder', u'trick', u'picnic', u'tabl', u'person', u'ride', u'skateboard', u'picnic', u'tabl', u'crowd', u'watch']
skateboard put show use picnic tabl stage skateboard pull trick top picnic tabl man ride skateboard top tabl skate boarder trick picnic tabl person ride skateboard picnic tabl crowd watch
[u'woman', u'walk', u'street', u'doorway', u'woman', u'walk', u'past', u'doorway', u'sidewalk', u'woman', u'talk', u'cell', u'phone', u'check', u'watch', u'woman', u'talk', u'cell', u'phone', u'check', u'watch', u'walk', u'sidewalk', u'woman', u'wear', u'blue', u'phone', u'walk', u'sidewalk', u'front', u'build', u'black', u'planter', u'side', u'entranc']
woman walk street doorway woman walk past doorway sidewalk woman talk cell phone check watch woman talk cell phone check watch walk sidewalk woman

In [10]:
vectorizer = CountVectorizer(min_df = 8, stop_words='english',analyzer='word', ngram_range=(1, 1))
vectorizer_p = CountVectorizer(min_df = 8, stop_words='english',analyzer='word', ngram_range=(1, 1))

vectorizer.fit(all_lines)
train_vectors = vectorizer.transform(all_lines)


vectorizer_p.fit(all_lines_p)
train_vectors_p = vectorizer_p.transform(all_lines_p)

In [11]:
print "features: " + str(len(vectorizer.vocabulary_))
print "features: " + str(len(vectorizer_p.vocabulary_))

features: 1452
features: 1664


In [12]:
test_vectors = vectorizer.transform(all_lines_test)
test_vectors_p = vectorizer_p.transform(all_lines_test_p)

# Load FC1000

In [13]:
fc1000_train_raw = pd.read_csv(filepath_or_buffer="./features_train/features_resnet1000_train.csv",header=None)
fc1000_train_raw[0] = fc1000_train_raw[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
fc1000_train_raw_sorted = fc1000_train_raw.sort_values(by=[0])
fc1000_train_img = fc1000_train_raw_sorted[0].values
fc1000_train = fc1000_train_raw_sorted[list(range(1,1001))].values

fc1000_test_raw = pd.read_csv(filepath_or_buffer="./features_test/features_resnet1000_test.csv",header=None)
fc1000_test_raw[0] = fc1000_test_raw[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
fc1000_test_raw_sorted = fc1000_test_raw.sort_values(by=[0])
fc1000_test_img = fc1000_test_raw_sorted[0].values
fc1000_test = fc1000_test_raw_sorted[list(range(1,1001))].values

# Load Pool5

In [14]:
pool5_train_raw = pd.read_csv(filepath_or_buffer="./features_train/features_resnet1000intermediate_train.csv",header=None)
pool5_train_raw[0] = pool5_train_raw[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
pool5_train_raw_sorted = pool5_train_raw.sort_values(by=[0])
pool5_train_img = pool5_train_raw_sorted[0].values
pool5_train = pool5_train_raw_sorted[list(range(1,2049))].values

pool5_test_raw = pd.read_csv(filepath_or_buffer="./features_test/features_resnet1000intermediate_test.csv",header=None)
pool5_test_raw[0] = pool5_test_raw[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
pool5_test_raw_sorted = pool5_test_raw.sort_values(by=[0])
pool5_test_img = pool5_test_raw_sorted[0].values
pool5_test = pool5_test_raw_sorted[list(range(1,2049))].values

In [15]:
print pool5_train.shape
print pool5_test.shape

(10000, 2048)
(2000, 2048)


# FC1000

In [16]:
train_data = fc1000_train
train_lable = postprocessing(train_vectors.toarray())
predict_data  = fc1000_test
test_lable = postprocessing(test_vectors.toarray())
NN_input_1 = test_lable

In [17]:
predict_tags = []
print "start: " + get_time_stamp()
clf_1 = Ridge(alpha = 30, tol=0.0001)
clf_1.fit(train_data, train_lable)
pre_result_1 = clf_1.predict(predict_data)
print "end: " + get_time_stamp()


2018-12-06_18-13-48
start: 2018-12-06_18-13-48
2018-12-06_18-13-50
end: 2018-12-06_18-13-50


# Pool5

In [18]:
train_data_p = pool5_train
train_lable_p = postprocessing(train_vectors_p.toarray())
predict_data_p  = pool5_test
test_lable_p = postprocessing(test_vectors_p.toarray())
NN_input_2 = test_lable_p

In [73]:
print "start: " + get_time_stamp()
#normalize=True

#rcv.fit(train_data_p, train_lable_p)
clf_2 = Ridge(alpha = 100)

clf_2.fit(train_data_p, train_lable_p)
pre_result_2 = clf_2.predict(predict_data_p)
print "end: " + get_time_stamp()


2018-12-06_19-11-22
start: 2018-12-06_19-11-22
2018-12-06_19-11-25
end: 2018-12-06_19-11-25


# Tag Bag of Words 

In [21]:
train_data_t = postprocessing(train_vectors.toarray())
train_lable_t = train_tag_vectors.toarray()
predict_data_t  = postprocessing(test_vectors.toarray())
test_lable_t = test_tag_vectors.toarray()
NN_input_3 = test_lable_t
train_data_t = sparse.csr_matrix(train_data_t)
predict_data_t = sparse.csr_matrix(predict_data_t)

In [22]:
predict_tags = []
for i in range(len(tag_vectorizer.vocabulary_)):
    #print str(i) + ": " + get_time_stamp()
    y = train_lable_t[:,i]
    clf = RandomForestRegressor(random_state=0, n_estimators=5, max_depth = 10)
    clf.fit(train_data_t, y)
    tag = clf.predict(predict_data_t)
    predict_tags.append(tag)
predict_tags_trans = np.array(predict_tags).T
pre_result_3 = predict_tags_trans

KeyboardInterrupt: 

In [26]:
import sklearn
def get_distacne_top_20_and_value(NN_input,pre_result,numbers,metric_method):
    #euclidean cosine jaccard
    distance_matrix = sklearn.metrics.pairwise_distances(NN_input,pre_result,metric = metric_method)
    # bb = sklearn.metrics.pairwise_distances(pre_result_2,NN_input_2,metric='euclidean')
    all_rank = []
    all_scores = []
    for line in distance_matrix:
        all_rank_row = []
        all_rank_scores = []
        heap =[]
        for key,value in enumerate(line):
            heappush(heap,(value,key))
        for i in range(numbers):
            value, key = heappop(heap)
            all_rank_row.append(key)
            all_rank_scores.append(value)
        all_rank.append(all_rank_row)
        all_scores.append(all_rank_scores)
    return all_rank, all_scores


# Get Result

In [ ]:
indices_cos_1,cosines_1 = get_distacne_top_20_and_value(NN_input_1, pre_result_1,20,'cosine')
indices_l2_1,l2_1 = get_distacne_top_20_and_value(NN_input_1, pre_result_1,20,'euclidean')

In [74]:
indices_cos_2,cosines_2 = get_distacne_top_20_and_value(NN_input_2, pre_result_2,20,'cosine')
indices_l2_2,l2_2 = get_distacne_top_20_and_value(NN_input_2, pre_result_2,20,'euclidean')

In [ ]:
indices_cos_3,cosines_3 = get_distacne_top_20_and_value(NN_input_3, pre_result_3,20,'cosine')
indices_l2_3,l2_3 = get_distacne_top_20_and_value(NN_input_3, pre_result_3,20,'euclidean')

In [ ]:
def get_average_cos_rank(indices_cos_1,cosines_1,indices_cos_2,cosines_2, indices_cos_3,cosines_3):
    ret = []

    for i in range(len(indices_cos_1)):
        scores = {}
        cosines_1[i] = cosines_1[i] / np.linalg.norm(cosines_1[i])
        cosines_2[i] = cosines_2[i] / np.linalg.norm(cosines_2[i])
        cosines_3[i] = cosines_3[i] / np.linalg.norm(cosines_3[i])

        for j in range(len(indices_cos_1[0])):
            img1 = indices_cos_1[i][j]
            img2 = indices_cos_2[i][j]
            img3 = indices_cos_3[i][j]
            
            scores[img1] = (scores.get(img1,0) +  30*(1-cosines_1[i][j]))
            scores[img2] = (scores.get(img2,0) +  100* (1-cosines_2[i][j]))
            scores[img3] = (scores.get(img3,0) +  22*(1-cosines_3[i][j]))
   
                
        row_ret = []
        heap =[]
        #print scores
        for key, val in scores.items():
            heappush(heap,(-val,key))
        
        for k in range(20):
            val, key= heappop(heap)

            #print key
            if key not in row_ret:
                row_ret.append(key)
        ret.append(row_ret)
    return ret

In [ ]:
indices_ensembling = get_average_cos_rank(indices_cos_1,cosines_1,indices_cos_2,cosines_2, indices_cos_3,cosines_3)
print indices_ensembling[0]
print indices_cos_1[0]
print indices_cos_2[0]
print indices_cos_3[0]

In [ ]:
# # Cross Validation
# def Three_Fold_Cross_Validation(train_set,labels):
#     print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+" Start Three_Fold_Cross_Validation")
#     k_fold = KFold(len(train_set),n_folds=3, shuffle=True)
#     accuracy = []
#     show_confusion_matrix_flag = True
#     for train_indices, test_indices in k_fold:
#         train = train_set[train_indices]
#         lable = labels[train_indices]
#         test  = train_set[test_indices]
#         validate = labels[test_indices]
#         result = []
#         for i in range(len(test_indices)):
#             result.append(KNN_Classifier(k_for_knn,train,lable,test[i]))
#         count = 0
#         for i in range(len(result)):
#             if validate[i] == result[i]:
#                 count+=1
#         accuracy.append(count/len(result))
        
#         if show_confusion_matrix_flag:
            
#             show_confusion_matrix_flag = False
#             cm = confusion_matrix(validate, result,labels=range(10))
#             print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+" confusion_matrix")
#             print(cm)
            
        
#     accuracy = np.array(accuracy)
    
    
#     return accuracy.mean()

# Submission

In [29]:
def submission(indices, model):
    result_scv = []
    for i in range(len(indices)):
        result_string = ""
        for j in indices[i]:
            result_string+= (str(j)+".jpg ")

        string_temp = (str(i)+".txt",    result_string[0:len(result_string)-1])
        result_scv.append(string_temp)

    pd_data = pd.DataFrame(np.array(result_scv),columns=['Descritpion_ID','Top_20_Image_IDs'])
    pd_data.to_csv(get_time_stamp() +'_'+model+'_.csv',index=False)


In [ ]:
submission(indices_ensembling,"ensembling")
submission(indices_l2_1,"fc1000")
submission(indices_l2_3,"tag")

In [75]:
submission(indices_l2_2,"pool5")

2018-12-06_19-11-32
